Tutorial 07: Environmental Effects Analysis

This tutorial explores environmental effects in satellite radio astronomy
observations using the RSC-SIM framework. It provides both educational demonstrations
and realistic modeling scenarios, running both parts by default.

Educational Components:
1. DEM loading and processing with rasterio for terrain analysis
2. Elevation masking for space-to-ground interactions
3. Advanced atmospheric refraction models (Bennett's formula + enhanced models)
4. Water vapor modeling for high-frequency simulations (weather radiometry)
5. Limb refraction effects for space-to-space interactions
6. Terrestrial antenna pointing limitations and mechanical constraints
7. Integrated line-of-sight calculations with DEM ray tracing and atmospheric effects


In [ ]:
import sys
import os
import time
import numpy as np
import pandas as pd
import warnings
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Add the src directory to the Python path
sys.path.insert(0, os.path.join(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))), 'src'))

from radio_types import Antenna, Instrument, Constellation, Trajectory, Observation  # noqa: E402
from astro_mdl import (  # noqa: E402
    estim_casA_flux, power_to_temperature, temperature_to_power, antenna_mdl_ITU, estim_temp
)
from sat_mdl import (  # noqa: E402
    sat_link_budget_vectorized,
    # Environmental effects functions
    calculate_comprehensive_environmental_effects_vectorized
)
from obs_mdl import model_observed_temp_with_atmospheric_refraction_vectorized  # noqa: E402
from env_mdl import AdvancedEnvironmentalEffects  # noqa: E402
import antenna_pattern  # noqa: E402

warnings.filterwarnings('ignore')

Create demo antenna and satellite trajectory.

In [ ]:
def create_demo_antenna():
    """Create a demo antenna for the Westford site"""
    # Simple antenna pattern (constant gain for demo)
    alphas = np.linspace(0, 180, 19)
    betas = np.linspace(0, 359, 37)  # Avoid 360 to prevent wrap-around issues
    gains = np.ones((len(alphas), len(betas))) * 0.5  # 50% efficiency

    # Create DataFrame for antenna pattern
    alpha_grid, beta_grid = np.meshgrid(alphas, betas, indexing='ij')
    gain_data = {
        'alphas': alpha_grid.flatten(),
        'betas': beta_grid.flatten(),
        'gains': gains.flatten()
    }
    gain_df = pd.DataFrame(gain_data)

    # Create antenna using from_dataframe method
    antenna = Antenna.from_dataframe(gain_df, rad_eff=0.5, valid_freqs=(10e9, 12e9))
    return antenna


def create_demo_satellite_trajectory():
    """Create a demo satellite trajectory"""
    # Create a simple satellite trajectory
    times = np.arange(0, 3600, 60)  # 1 hour, 1 minute intervals

    # Simple circular orbit simulation
    orbit_altitude = 550000  # meters (typical LEO)
    earth_radius = 6371000
    orbit_radius = earth_radius + orbit_altitude

    # Satellite passes from south to north
    # Use elevation angles (0° to 90°) instead of declinations
    elevation_angles = np.linspace(0, 90, len(times))  # degrees - horizon to zenith
    azimuths = np.linspace(180, 0, len(times))  # degrees
    ranges = np.full(len(times), orbit_radius)

    # Convert times to datetime objects
    start_time = datetime.now()
    datetime_times = [start_time + timedelta(seconds=int(t)) for t in times]

    traj_df = pd.DataFrame({
        'times': datetime_times,
        'azimuths': azimuths,
        'elevations': elevation_angles,
        'distances': ranges
    })

    trajectory = Trajectory(traj=traj_df)
    return trajectory

Environment effects with demo data.

In [ ]:
def demonstrate_advanced_environmental_effects():
    """
    Demonstrate advanced propagation and environmental effects using demo data.

    This function uses:
    - Demo antenna patterns (constant gain)
    - Demo satellite trajectories (2025-09-10 time range)
    - Demo atmospheric conditions
    - Demo environmental effects parameters

    This is for educational purposes to show how environmental effects work
    without requiring real data files.
    """
    print("=" * 80)
    print("ADVANCED PROPAGATION & ENVIRONMENTAL EFFECTS DEMONSTRATION")
    print("=" * 80)

    # Westford antenna coordinates
    westford_lat = 42.6129479883915  # degrees
    westford_lon = -71.49379366344017  # degrees
    westford_elevation = 86.7689687917009  # meters above sea level

    # Atmospheric conditions (typical for Westford, MA)
    temperature = 288.15  # K (15°C)
    pressure = 101325  # Pa (1 atm)
    humidity = 60.0  # % (moderate humidity)

    print("\nAntenna Location: Westford, MA")
    print(f"  Latitude: {westford_lat}°")
    print(f"  Longitude: {westford_lon}°")
    print(f"  Elevation: {westford_elevation} m")
    print(f"  Temperature: {temperature:.1f} K ({temperature-273.15:.1f}°C)")
    print(f"  Pressure: {pressure/1000:.1f} kPa")
    print(f"  Humidity: {humidity:.1f}%")

    # Initialize environmental effects modeling
    script_dir = os.path.dirname(os.path.abspath('__file__'))
    dem_file = os.path.join(
        script_dir, "..", "research_tutorials", "data",
        "USGS_OPR_MA_CentralEastern_2021_B21_be_19TBH294720.tif"
    )
    environment = AdvancedEnvironmentalEffects(
        dem_file, westford_lat, westford_lon, westford_elevation,
        temperature, pressure, humidity)

    # Test specific satellite positions with atmospheric effects
    # Fix: Use proper satellite range (altitude + Earth radius)
    earth_radius = 6371000  # meters
    satellite_altitude = 550000  # meters (typical LEO altitude)
    satellite_range = earth_radius + satellite_altitude  # Total range from antenna to satellite

    test_positions = [
        (30, 0, satellite_range),    # High elevation, due north
        (10, 0, satellite_range),    # Medium elevation, due north
        (5, 0, satellite_range),     # Low elevation, due north
        (30, 90, satellite_range),   # High elevation, due east
        (30, 180, satellite_range),  # High elevation, due south
        (30, 270, satellite_range),  # High elevation, due west
        (2, 0, satellite_range),     # Very low elevation (should be blocked)
        (45, 45, satellite_range),   # High elevation, northeast
    ]

    print("\nAdvanced environmental effects results for test positions:")
    print("Alt(°)  Az(°)   Range(m)  Antenna  Elevation  LOS     Final  Apparent  WV Abs  Atm Loss")
    print("                                 Access   Masking  Visible  Elev(°)   (dB)    (dB)")
    print("-" * 85)

    for alt, az, rng in test_positions:
        antenna_access = environment.check_antenna_limitations(alt)
        elevation_ok = environment.check_elevation_masking(alt)
        is_visible, blocking_elev, atm_effects = environment.check_line_of_sight(alt, az, rng)
        final_masking, final_atm_effects = environment.apply_terrain_masking(alt, az, rng)

        apparent_elev = final_atm_effects.get('apparent_elevation', alt)
        wv_abs = final_atm_effects.get('water_vapor_absorption', 0.0)
        atm_loss = final_atm_effects.get('total_atmospheric_loss', 0.0)

        print(f"{alt:6.1f}  {az:6.1f}  {rng:8.0f}  {antenna_access!s:7s}  {elevation_ok!s:9s} "
              f"{is_visible!s:7s}  {final_masking:5.2f}  {apparent_elev:8.2f}  {wv_abs:6.3f}  {atm_loss:7.3f}")

    # Demonstrate with satellite trajectory
    print("\n" + "=" * 80)
    print("SATELLITE TRAJECTORY ANALYSIS")
    print("=" * 80)

    # Create demo satellite trajectory for analysis
    print("Creating demo satellite trajectory...")
    trajectory = create_demo_satellite_trajectory()

    # Analyze trajectory with comprehensive terrain masking
    print(f"\nAnalyzing {len(trajectory.traj)} trajectory points...")

    times = trajectory.traj['times']
    elevation_angles = trajectory.traj['elevations']
    azimuths = trajectory.traj['azimuths']
    ranges = trajectory.traj['distances']

    # Calculate comprehensive terrain masking factors
    masking_factors = []
    terrain_blocking_details = []
    visible_count = 0

    for i in range(len(times)):
        alt = elevation_angles.iloc[i]
        az = azimuths.iloc[i]
        rng = ranges.iloc[i]

        # Apply comprehensive environmental effects
        masking_factor, atm_effects = environment.apply_terrain_masking(alt, az, rng)
        masking_factors.append(masking_factor)

        # Get detailed blocking information
        antenna_ok = environment.check_antenna_limitations(alt)
        elevation_ok = environment.check_elevation_masking(alt)
        los_visible, blocking_elev, los_atm_effects = environment.check_line_of_sight(alt, az, rng)

        terrain_blocking_details.append({
            'antenna_ok': antenna_ok,
            'elevation_ok': elevation_ok,
            'los_visible': los_visible,
            'blocking_elev': blocking_elev
        })

        if masking_factor > 0.5:
            visible_count += 1

    masking_factors = np.array(masking_factors)
    visibility_percentage = (visible_count / len(times)) * 100

    # Summary statistics
    print("\n" + "=" * 80)
    print("TERRAIN MASKING ANALYSIS SUMMARY")
    print("=" * 80)

    # Calculate comprehensive blocking statistics
    antenna_blocked = sum(1 for detail in terrain_blocking_details if not detail['antenna_ok'])
    elevation_blocked = sum(1 for detail in terrain_blocking_details if not detail['elevation_ok'])
    terrain_blocked = sum(1 for detail in terrain_blocking_details if not detail['los_visible'])

    print(f"Total observation points: {len(times)}")
    print(f"Visible points: {visible_count}")
    print(f"Visibility percentage: {visibility_percentage:.1f}%")
    print(f"Blocked points: {len(times) - visible_count}")

    print("\nBlocking breakdown:")
    print(f"  Antenna limitations: {antenna_blocked} points ({antenna_blocked/len(times)*100:.1f}%)")
    print(f"  Elevation masking: {elevation_blocked} points ({elevation_blocked/len(times)*100:.1f}%)")
    print(f"  Terrain blocking: {terrain_blocked} points ({terrain_blocked/len(times)*100:.1f}%)")

    # Show trajectory details
    print("\nTrajectory details:")
    print(f"  Time range: {times.iloc[0]} to {times.iloc[-1]}")
    print(f"  Elevation range: {np.min(elevation_angles):.1f}° to {np.max(elevation_angles):.1f}°")
    print(f"  Azimuth range: {np.min(azimuths):.1f}° to {np.max(azimuths):.1f}°")
    print(f"  Range: {np.min(ranges)/1000:.1f} to {np.max(ranges)/1000:.1f} km")

    # Show detailed example points
    print("\nDetailed trajectory analysis (every 10th point):")
    print("Index  Alt(°)  Az(°)   Range(m)  Ant  Elev  LOS   Final")
    print("-" * 55)
    for i in range(0, len(times), max(1, len(times)//10)):
        alt = elevation_angles.iloc[i]
        az = azimuths.iloc[i]
        rng = ranges.iloc[i]
        mf = masking_factors[i]
        detail = terrain_blocking_details[i]

        ant_str = "✓" if detail['antenna_ok'] else "✗"
        elev_str = "✓" if detail['elevation_ok'] else "✗"
        los_str = "✓" if detail['los_visible'] else "✗"

        print(f"{i:5d}  {alt:6.1f}  {az:6.1f}  {rng:8.0f}  {ant_str:3s}  {elev_str:4s}  {los_str:3s}  {mf:6.2f}")

    # Environmental effects parameters
    print("\nEnvironmental effects parameters:")
    print(f"  Minimum elevation angle: {environment.min_elevation_angle}°")
    print(f"  Refraction coefficient: {environment.refraction_coefficient}")
    print(f"  Antenna mechanical limit: {environment.antenna_mechanical_limit}°")
    print(f"  Antenna elevation: {environment.antenna_elevation} m")
    print(f"  Surface temperature: {environment.temperature:.1f} K")
    print(f"  Surface pressure: {environment.pressure/1000:.1f} kPa")
    print(f"  Relative humidity: {environment.humidity:.1f}%")
    print(f"  Water vapor scale height: {environment.water_vapor_scale_height:.0f} m")

    # Demonstrate advanced atmospheric effects
    print("\n" + "=" * 80)
    print("ADVANCED ATMOSPHERIC EFFECTS ANALYSIS")
    print("=" * 80)

    elevation_angles = [1, 2, 5, 10, 15, 20, 30, 45, 60, 90]
    frequencies = [1e9, 11e9, 22e9, 94e9]  # Different frequency bands

    print("Atmospheric Refraction Analysis:")
    print("True Elev  Apparent Elev  Refraction  Atm Delay  WV Abs    WV Emiss  Total Loss")
    print("                              Correction    (m)      (dB)      (K)      (dB)")
    print("-" * 80)

    for true_elev in elevation_angles:
        (apparent_elev, atm_delay, wv_abs, wv_emiss, total_loss) = \
            environment.calculate_integrated_atmospheric_effects(true_elev, 11e9)
        correction = apparent_elev - true_elev
        print(f"{true_elev:10.1f}°  {apparent_elev:12.3f}°  {correction:10.3f}°  "
              f"{atm_delay:8.1f}  {wv_abs:8.3f}  {wv_emiss:8.1f}  {total_loss:8.3f}")

    print("\nWater Vapor Effects at Different Frequencies:")
    print("Frequency  True Elev  WV Absorption  WV Emission  Total Atm Loss")
    print("   (GHz)      (°)        (dB)          (K)          (dB)")
    print("-" * 60)

    for freq in frequencies:
        for true_elev in [5, 15, 30]:
            (apparent_elev, atm_delay, wv_abs, wv_emiss, total_loss) = \
                environment.calculate_integrated_atmospheric_effects(true_elev, freq)
            print(f"{freq/1e9:8.1f}  {true_elev:10.1f}°  {wv_abs:12.3f}  "
                  f"{wv_emiss:10.1f}  {total_loss:12.3f}")

    print("\nAtmospheric Profile Analysis:")
    print("Height (m)  Temperature (K)  Pressure (kPa)  Water Vapor (g/m³)")
    print("-" * 60)

    heights = [0, 1000, 2000, 5000, 10000, 15000, 20000]
    for height in heights:
        T, P, WV = environment.calculate_atmospheric_profile(height)
        print(f"{height:10.0f}  {T:14.1f}  {P/1000:12.1f}  {WV*1000:14.2f}")

    # Demonstrate antenna pointing limitations
    print("\n" + "=" * 80)
    print("ANTENNA POINTING LIMITATIONS")
    print("=" * 80)

    print("Common antenna pointing limitations:")
    print("  - Mechanical limits: typically 5-10° minimum elevation")
    print("  - Atmospheric effects: increased noise at low elevations")
    print("  - Terrain masking: local topography blocks line of sight")
    print("  - RFI sources: ground-based interference at low elevations")

    # Show impact of different minimum elevation angles
    min_elevations = [1, 3, 5, 10, 15]
    print("\nImpact of different minimum elevation angles:")
    print("Min Elev  Visible Points  Visibility %")
    print("-" * 35)

    test_elevations = np.linspace(0, 90, 91)
    for min_elev in min_elevations:
        environment.min_elevation_angle = min_elev
        visible_count = 0
        for elev_angle in test_elevations:
            if environment.check_elevation_masking(elev_angle):
                visible_count += 1
        visibility_pct = (visible_count / len(test_elevations)) * 100
        print(f"{min_elev:8.1f}°  {visible_count:12d}  {visibility_pct:11.1f}%")

    # Demonstrate limb refraction effects (space-to-space)
    print("\n" + "=" * 80)
    print("LIMB REFRACTION EFFECTS (SPACE-TO-SPACE)")
    print("=" * 80)

    print("Limb refraction effects for space-to-space interactions:")
    print("  - Occurs when satellite signals pass through Earth's atmosphere")
    print("  - Most significant for grazing incidence angles")
    print("  - Can cause signal bending and attenuation")
    print("  - Important for satellite-to-satellite communications")

    # Simple limb refraction model
    grazing_angles = [0.1, 0.5, 1.0, 2.0, 5.0]  # degrees
    print("\nLimb refraction effects at different grazing angles:")
    print("Grazing Angle  Refraction Effect  Signal Bending")
    print("-" * 45)

    for angle in grazing_angles:
        # Apply limb refraction model
        limb_refraction, signal_bending = environment.apply_limb_refraction(angle)
        print(f"{angle:11.1f}°  {limb_refraction:14.2f}°  {signal_bending:12.3f}°")

    print("\nAnalysis complete!")
    print("\nKey findings:")
    print(f"  - {visibility_percentage:.1f}% of satellite trajectory is visible")
    print("  - Elevation masking blocks observations below minimum threshold")
    print("  - Advanced atmospheric refraction significantly affects low elevation observations")
    print("  - Water vapor absorption increases with frequency and humidity")
    print("  - Atmospheric-terrain interactions influence local atmospheric conditions")
    print("  - Integrated atmospheric effects provide comprehensive signal path modeling")
    print("  - Antenna pointing limitations further reduce observable sky area")
    print("  - Limb refraction affects space-to-space communications")
    print("  - Environmental effects modeling enables realistic radio astronomy simulations")


demonstrate_advanced_environmental_effects()